In [5]:
'''
Benjamin Barnett
8/8/18
https://projecteuler.net/problem=142

Find the smallest x + y + z with integers x > y > z > 0
such that x + y, x − y, x + z, x − z, y + z, y − z are all perfect squares.
'''

import pandas as pd
import numpy as np
from primesieve import *
import itertools
from numba import vectorize
import psutil
import sys

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.max_rows    = 1000
pd.options.display.max_columns = 1000

@vectorize(["uint32(uint32,uint32)"], target='cuda')
def vector_add(a,b):
    return a + b

@vectorize(["uint32(uint32,uint32)"], target='cuda')
def vector_subtract(a,b):
    return a - b

d_type = np.uint32                     # data type of numbers being used
d_size = np.iinfo(d_type).bits      # size of the d_type in bytes
max_dtype_val = np.iinfo(d_type).max   # maximum value for dtype

min_val = 1
max_val = 2

squares = []
i = min_val
while i**2 <= max_val*2:
    squares.append(i**2)
    i += 1
squares = set(squares)

i = 1
memory_needed = (d_size*(max_val-min_val))*9*((max_val-min_val)**3) + (len(squares) * d_size)

while memory_needed < psutil.virtual_memory().available:
    max_val += 1
    memory_needed = (d_size*(max_val-min_val))*9*((max_val-min_val)**3) + (len(squares) * d_size)
    squares = []
    i = min_val
    while i**2 <= max_val*2:
        squares.append(i**2)
        i += 1
    squares = set(squares)

max_val -= 1
increment = max_val

print("Increment: %i" % max_val)

def is_square(n):
    return n in squares

while True:
    s1 = None
    s2 = None
    s3 = None
    s4 = None
    s5 = None
    s6 = None
    
    x_s = None
    y_s = None
    z_s = None
    
    if max_val > max_dtype_val: # maxiumum dtype value
        print("%i is too large of a value for %s (max val: %i)" % (max_val,str(d_type).split('\'')[1]),max_dtype_val)
        sys.exit()

    memory_needed = (d_size*(max_val-min_val))*9*((max_val-min_val)**3) + (len(squares) * d_size)
    if memory_needed > psutil.virtual_memory().available:
        print("Memory Needed (%s GB) > Memory Available (%s GB)" % ("{0:.2f}".format(memory_needed * 10**-9),"{0:.2f}".format(psutil.virtual_memory().available * 10**-9)))
        sys.exit()
    
    s1 = np.zeros((max_val-min_val)**3,dtype=d_type)
    s2 = np.zeros((max_val-min_val)**3,dtype=d_type)
    s3 = np.zeros((max_val-min_val)**3,dtype=d_type)
    s4 = np.zeros((max_val-min_val)**3,dtype=d_type)
    s5 = np.zeros((max_val-min_val)**3,dtype=d_type)
    s6 = np.zeros((max_val-min_val)**3,dtype=d_type)

    x_s = []
    y_s = []
    z_s = []

    for x in range(min_val,max_val+1):
        for y in range(min_val,max_val+1):
            for z in range(min_val,max_val+1):
                x_s.append(d_type(x))
                y_s.append(d_type(y))
                z_s.append(d_type(z))

    s1 = vector_add(x,y)
    s2 = vector_subtract(x,y)
    s3 = vector_add(x,z)
    s4 = vector_subtract(x,z)
    s5 = vector_add(y,z)
    s6 = vector_subtract(y,z)

    for i,v in enumerate(s1):
        if is_square(s1[i]):
            if is_square(s2[i]):
                if is_square(s3[i]):
                    if is_square(s4[i]):
                        if is_square(s5[i]):
                            if is_square(s6[i]):
                                print(x_s[i],y_s[i],z_s[i]," = ",x_s[i]+y_s[i]+z_s[i])
                                sys.exit()
    
    min_val += increment - 1
    max_val += increment - 1
    
    squares = []
    i = min_val
    while i**2 <= max_val*2:
        squares.append(i**2)
        i += 1
    squares = set(squares)

Increment: 86
Memory Needed (15.03 GB) > Memory Available (15.02 GB)


SystemExit: 

/home/ben/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
